######**PubMed Preprocessing and Gene,Disease Masking**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/NLP/Genomatics Package/BioBert/biobert-pytorch/relation-extraction')

In [ ]:
#We extract the pubmed document in BioCJSON format
import urllib3
import json
import csv
import requests
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from itertools import combinations

pmcid = 'PMC2837563'

http = urllib3.PoolManager()

r = http.request('GET', f'https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson?pmcids={pmcid}')
data = json.loads(r.data.decode('utf-8'))
#data = json.dumps(data, indent=4)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [ ]:
#os.chdir('BioBert/biobert-pytorch/datasets/RE/GAD')

In [ ]:
f1 = open('testset/test.tsv', 'w')
tsv_writer_1 = csv.writer(f1, delimiter='\t')
tsv_writer_1.writerow(['index', 'sentence'])
f2 = open('pubtator_outputs/pub_original_sentences.tsv', 'w')
tsv_writer_2 = csv.writer(f2, delimiter='\t')
tsv_writer_2.writerow(['index', 'sentence', 'entity_1', 'entity_2'])

34

In [ ]:
sentence_index = 0
sentence_entities = {}
for i in data['passages']:
  if i['infons']['section_type'] != 'TABLE':
    #filter the table segment
    text = i['text']
    offset = i['offset']
    annotations = i['annotations']

    annotations = sorted(annotations, key = lambda x: x['locations'][0]['offset'])
    #Filter to only include gene-disease annotations
    annotations = [annotation for annotation in annotations if ((annotation['infons']['type']=='Gene') or (annotation['infons']['type']=='Disease'))]
    #List all possible combinations of annotations
    annots_combinations = list(combinations(annotations, 2))
    #Filter combinations to only include gene-disease combinations
    annots_combinations = [annots for annots in annots_combinations if annots[0]['infons']['type'] != annots[1]['infons']['type']]


    #processing sentences
    sentences = text.split('. ')
    sentence_offset = {}
    sentence_len = {}
    prev_sent_offset = offset

    for sentence in sentences:

      sentence_offset[sentence] = prev_sent_offset
      current_sentence_offset = prev_sent_offset
      sentence_len[sentence] = len(sentence)
      current_sentence_len = len(sentence)
      prev_sent_offset += len(sentence) + 2
      

      #Point to note: Duplicate sentences for as many combinations as present. 
      for annots in annots_combinations:
        difference = 0
        current_sentence = sentence
        #sort the tuple
        annots = sorted(annots, key = lambda x: x['locations'][0]['offset'])

        entity_1 = annots[0]
        entity_2 = annots[1]
        
        entity_1_offset = entity_1['locations'][0]['offset']
        entity_2_offset = entity_2['locations'][0]['offset']

        entity_1_dist = entity_1_offset - current_sentence_offset
        entity_2_dist = entity_2_offset - current_sentence_offset

        if (0 <= entity_1_dist <= ((current_sentence_len - len(entity_1['text'])) + 1)) and (0 <= entity_2_dist <= ((current_sentence_len - len(entity_2['text'])) + 1)):
          #the pair of annotations fall within the sentence
          sentence_entities[sentence_index] = (entity_1['text'], entity_2['text'])

          entity_1_type = entity_1['infons']['type']
          entity_1_length = entity_1['locations'][0]['length']
          temp = '@'+ entity_1_type +'$'
          entity_1_final_off = entity_1_dist 
          current_sentence = current_sentence[:entity_1_final_off] + "@" + entity_1_type + "$" + current_sentence[(entity_1_final_off + entity_1_length):]
          difference += (entity_1_length - len(temp))

          entity_2_type = entity_2['infons']['type']
          entity_2_length = entity_2['locations'][0]['length']
          temp = '@'+ entity_2_type +'$'
          entity_2_final_off = entity_2_dist - (difference)
          current_sentence = current_sentence[:entity_2_final_off] + "@" + entity_2_type + "$" + current_sentence[(entity_2_final_off + entity_2_length):]
          difference += (entity_2_length - len(temp))
          tsv_writer_1.writerow([sentence_index, current_sentence])
          tsv_writer_2.writerow([sentence_index, sentence, entity_1['text'], entity_2['text']])
          sentence_index += 1

######**BioBERT Training**

In [ ]:
!pip install transformers==3.0.0

     |████████████████████████████████| 754 kB 5.4 MB/s 
     |████████████████████████████████| 3.0 MB 54.3 MB/s 
     |████████████████████████████████| 1.2 MB 27.5 MB/s 
     |████████████████████████████████| 880 kB 54.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=03e8585623dfde8a3f96f9424cc9abe6c98d88efe8d101c630a8b3a390aa6c1f
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [ ]:
#!git clone https://github.com/dmis-lab/biobert-pytorch.git

In [ ]:
#os.chdir('/content/drive/MyDrive/NLP/Genomatics Package/BioBert/biobert-pytorch')

In [ ]:
!bash ./download.sh

BIOBERT_DATA not set; downloading to default path ('data').
--2022-05-14 03:51:56--  https://docs.google.com/uc?export=download&confirm=t&id=1cGqvAm9IZ_86C4Mj7Zf-w9CFilYVDl8j
Resolving docs.google.com (docs.google.com)... 108.177.119.101, 108.177.119.102, 108.177.119.138, ...
Connecting to docs.google.com (docs.google.com)|108.177.119.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-20-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qu9qdum5f8s73039mq83p7pc9t15mela/1652500275000/13799006341648886493/*/1cGqvAm9IZ_86C4Mj7Zf-w9CFilYVDl8j?e=download [following]
--2022-05-14 03:51:56--  https://doc-10-20-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qu9qdum5f8s73039mq83p7pc9t15mela/1652500275000/13799006341648886493/*/1cGqvAm9IZ_86C4Mj7Zf-w9CFilYVDl8j?e=download
Resolving doc-10-20-docs.googleusercontent.com (doc-10-20-docs.googleusercontent.com)... 173.194.69.132, 2a00:1450:4013:c04::

In [ ]:
!bash ./preprocess.sh

*****  euadr  Preprocessing Start *****
*****  euadr  Preprocessing Done *****
*****  GAD  Preprocessing Start *****
*****  GAD  Preprocessing Done *****


In [ ]:
%env SAVE_DIR=./output
%env DATA="GAD"
%env SPLIT="1"
%env DATA_DIR=../datasets/RE/GAD/1
%env ENTITY=${DATA}-${SPLIT}

%env MAX_LENGTH=128
%env BATCH_SIZE=32
%env NUM_EPOCHS=3
%env SAVE_STEPS=1000
%env SEED=1

env: SAVE_DIR=./output
env: DATA="GAD"
env: SPLIT="1"
env: DATA_DIR=../datasets/RE/GAD/1
env: ENTITY=${DATA}-${SPLIT}
env: MAX_LENGTH=128
env: BATCH_SIZE=32
env: NUM_EPOCHS=3
env: SAVE_STEPS=1000
env: SEED=1


In [ ]:
!echo DATA_DIR

DATA_DIR


In [ ]:
!python run_re.py \
    --task_name SST-2 \
    --config_name bert-base-cased \
    --data_dir ${DATA_DIR} \
    --model_name_or_path dmis-lab/biobert-base-cased-v1.1 \
    --max_seq_length ${MAX_LENGTH} \
    --num_train_epochs ${NUM_EPOCHS} \
    --per_device_train_batch_size ${BATCH_SIZE} \
    --save_steps ${SAVE_STEPS} \
    --seed ${SEED} \
    --do_train \
    --do_predict \
    --learning_rate 5e-5 \
    --output_dir ${SAVE_DIR}/${ENTITY} \
    --overwrite_output_dir

05/14/2022 05:11:34 - INFO - transformers.training_args -   PyTorch: setting up devices
05/14/2022 05:11:34 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
05/14/2022 05:11:35 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/dmis-lab/biobert-base-cased-v1.1/config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp7mihet3v
Downloading: 100% 313/313 [00:00<00:00, 272kB/s]
05/14/2022 05:11:35 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/dmis-lab/biobert-base-cased-v1.1/config.json in cache at /root/.cache/torch/transformers/efc161c68d589c7960ab5463ed06a47d75d1ec73b2c31938de0ff797f76892dd.d7a3af18ce3a2ab7c0f48f04dc8daff45ed9a3ed333b9e9a79d012a0dedf87a6
05/14/2022 05:11:35 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/efc161

######**Applying BioBERT for our Dataset**

In [ ]:
%env SAVE_DIR=./output/test
%env DATA="GAD"
%env SPLIT="1"
%env DATA_DIR=./testset
%env ENTITY=${DATA}-${SPLIT}

%env MAX_LENGTH=128
%env BATCH_SIZE=32
%env NUM_EPOCHS=3
%env SAVE_STEPS=1000
%env SEED=1

env: SAVE_DIR=./output/test
env: DATA="GAD"
env: SPLIT="1"
env: DATA_DIR=./testset
env: ENTITY=${DATA}-${SPLIT}
env: MAX_LENGTH=128
env: BATCH_SIZE=32
env: NUM_EPOCHS=3
env: SAVE_STEPS=1000
env: SEED=1


In [ ]:
!python run_re.py --task_name SST-2 --config_name bert-base-cased --model_name_or_path dmis-lab/biobert-base-cased-v1.1 \
        --do_predict --data_dir ${DATA_DIR} \
        --output_dir ${SAVE_DIR} \
        --overwrite_output_dir


05/14/2022 05:18:53 - INFO - transformers.training_args -   PyTorch: setting up devices
05/14/2022 05:18:53 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
05/14/2022 05:18:53 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/dmis-lab/biobert-base-cased-v1.1/config.json from cache at /root/.cache/torch/transformers/efc161c68d589c7960ab5463ed06a47d75d1ec73b2c31938de0ff797f76892dd.d7a3af18ce3a2ab7c0f48f04dc8daff45ed9a3ed333b9e9a79d012a0dedf87a6
05/14/2022 05:18:53 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_he

######**Unmasking Bio Medical terms of interest**

In [ ]:
original_sentences = pd.read_csv('pubtator_outputs/pub_original_sentences.tsv',sep='\t')
predictions = pd.read_csv('output/test/test_results.txt', sep='\t')

final_re_output = pd.merge(original_sentences, predictions, on ='index', how='left')
final_re_output

,index,sentence,entity_1,entity_2,prediction
0,0,Activating K-Ras mutations outwith 'hotspot' c...,K-Ras,colorectal tumours,1
1,1,Activating K-Ras mutations outwith 'hotspot' c...,K-Ras,cancer,1
2,2,Response to EGFR-targeted therapies in colorec...,EGFR,colorectal cancer,0
3,3,Response to EGFR-targeted therapies in colorec...,colorectal cancer,Kirsten-Ras,1
4,4,Response to EGFR-targeted therapies in colorec...,colorectal cancer,K-Ras,0
...,...,...,...,...,...
216,6,Colorectal tumours (n=106) were screened for a...,Colorectal tumours,K-Ras,1
217,7,Four additional K-Ras mutations (Leu19Phe (1 o...,K-Ras,tumours,1
218,8,We additionally identified a new K-Ras gene am...,K-Ras,tumours,1
219,9,The identification of mutations outwith previo...,K-Ras,colorectal tumours,1


In [ ]:
final_re_output.to_csv('pubtator_outputs/final_relation_extraction_output.csv')